<a href="https://colab.research.google.com/github/lastinm/ml_hw_notebooks/blob/main/%D0%A5%D1%80%D0%B0%D0%BD%D0%B5%D0%BD%D0%B8%D0%B5%2C_%D0%B8%D0%B7%D0%B2%D0%BB%D0%B5%D1%87%D0%B5%D0%BD%D0%B8%D0%B5_%D0%B8_%D0%BE%D0%B1%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%BA%D0%B0_%D0%B1%D0%BE%D0%BB%D1%8C%D1%88%D0%B8%D1%85_%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85_%D0%9B%D0%B0%D0%B1%D0%BE%D1%80%D0%B0%D1%82%D0%BE%D1%80%D0%BD%D0%B0%D1%8F_%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0_%E2%84%964.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Задание 1. Бинарная классификация (BinaryClassification)




## 1. Сгенерировать подходящие исходные данные для проведения обучения.

In [1]:
# Импорт датасета
from sklearn.datasets import make_classification


X, y = make_classification(
    n_samples = 1000,
    n_features = 15,
    n_informative = 12,
    n_redundant=3,
    n_clusters_per_class=2
)

In [2]:
# Функция для записи данных в файл
def to_file(path, X, y):
    lines = []
    for i in range(len(y)):
      features = []
      for j in range(len(X[0])):
        features.append(f"{j+1}:{X[i][j]}")

      lines.append(f"{y[i]} {' '.join(features)}")

    with open(path, "w") as file:
      file.write('\n'.join(lines))

In [3]:
# Запись датасета в файл
to_file("/content/classification_data", X, y)

## 2. Провести бинарную классификацию с помощью pyspark.ml / pyspark.mllib

In [11]:
# Импорт необходимых библиотек
from pyspark.sql import SparkSession
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.classification import LogisticRegressionWithLBFGS
from pyspark.mllib.evaluation import BinaryClassificationMetrics

In [12]:
# Загрузка данных в Spark
spark = SparkSession.builder.appName('Lab 4').getOrCreate()
data = spark.read.format("libsvm").load("/content/classification_data")
data.head()

Row(label=1.0, features=SparseVector(15, {0: -2.83, 1: -1.9738, 2: 0.9619, 3: -3.6214, 4: 1.1093, 5: 2.0564, 6: 2.6481, 7: 9.0334, 8: 6.3909, 9: 1.2437, 10: -0.9959, 11: 10.1288, 12: -3.0733, 13: 4.5553, 14: -0.1862}))

In [13]:
# преобразуем DataFrame в RDD (Resilient Distributed Dataset)
dataset = data.rdd.map(lambda row: LabeledPoint(row.label, list(row.features))).collect()

# создаем новый RDD, параллельно распараллеливая (распределяя) список dataset
dataset = spark.sparkContext.parallelize(dataset)

In [14]:
# разобьем датасет на обучающую и тестовую части
training, test = dataset.randomSplit([0.8, 0.2])

In [15]:
# обучаем модель на обучающем датасете
model = LogisticRegressionWithLBFGS.train(training)

## 3. Провести расчет метрик

In [16]:
# оцениваем качество модели на тестовом датасете
predictions = test.map(lambda row: (float(model.predict(row.features)), row.label))

In [17]:
# вычисляем метрики
metrics = BinaryClassificationMetrics(predictions)
(metrics.areaUnderPR, metrics.areaUnderROC)

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:158: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


(0.7386484255834849, 0.7396973742768135)

## 4. Сохранить полученные значения метрик в файл

In [18]:
import json

# Создаем словарь с метриками
metrics_dict = {
    "Area Under PR": metrics.areaUnderPR,
    "Area Under ROC": metrics.areaUnderROC
}

# Запись метрик в JSON файл
with open("/content/metrics.json", 'w') as json_file:
    json.dump(metrics_dict, json_file, indent=4)

# Задание 2. Кластеризация (KMeans)

## 1. Сгенерировать подходящие исходные данные для проведения обучения

In [19]:
# Импорт датасета
from sklearn.datasets import make_classification

# Генерация датасета
X, y = make_classification(
    n_samples = 1000,
    n_features = 15,
    n_informative = 12,
    n_redundant=3,
    n_clusters_per_class=2,
    n_classes= 5
)
# Запись датасета в файл
to_file("/content/clustering", X, y)

In [20]:
# Загрузка данных в Spark
data = spark.read.format("libsvm").load("clustering")
data.head()

Row(label=2.0, features=SparseVector(15, {0: -1.4251, 1: -0.725, 2: -3.1891, 3: -5.0098, 4: 1.097, 5: -2.9258, 6: -2.3779, 7: -3.5479, 8: 0.3267, 9: -0.7171, 10: -2.1423, 11: -0.3866, 12: 1.8101, 13: 0.6878, 14: -4.3943}))

## 2. Провести кластеризацию с помощью pyspark.ml / pyspark.mllib

In [22]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

# Инициализация модели
kmeans = KMeans().setK(5)

# Обучение
model = kmeans.fit(data)

## 3. Вывести и сохранить в файл полученные центры кластеров

In [23]:
# Предсказание
predictions = model.transform(data)
predictions.head()

Row(label=2.0, features=SparseVector(15, {0: -1.4251, 1: -0.725, 2: -3.1891, 3: -5.0098, 4: 1.097, 5: -2.9258, 6: -2.3779, 7: -3.5479, 8: 0.3267, 9: -0.7171, 10: -2.1423, 11: -0.3866, 12: 1.8101, 13: 0.6878, 14: -4.3943}), prediction=2)

In [24]:
# Оценка
evaluation = ClusteringEvaluator().evaluate(predictions)
evaluation

0.1859211909316538

In [25]:
# Получение центроидов
centers = model.clusterCenters()
centers

[array([-0.79082767, -2.53399565,  0.38841928, -0.1891949 ,  0.50664294,
         1.74080537, -0.08098493,  1.43350291, -1.93940499,  0.81150963,
         6.30493828,  0.34311976,  0.9301877 ,  1.90739504,  5.40326834]),
 array([-1.48875636, -5.24265486, -1.76907089,  1.39608969,  0.93134151,
        -0.44666482,  0.79891211,  0.33677465, -0.64152568,  0.2163354 ,
         3.53400041,  0.28712882,  0.833132  ,  0.59696798, -1.2237118 ]),
 array([-1.02847958, -0.76552181, -1.01675082,  0.77127375,  0.12899759,
        -1.28377132,  0.34212066, -0.33934406,  1.40682189, -1.07866815,
        -2.26197756,  0.19177865, -0.2889259 , -1.10387814, -5.0331478 ]),
 array([ 0.78548961,  0.49374225, -0.26184777, -0.49353094, -0.43972035,
         0.52078801, -0.34981424,  0.3217464 , -0.43254287, -0.28021695,
         0.28106494, -0.32506322,  0.12317938,  0.9068741 ,  1.8179935 ]),
 array([ 1.87991748,  5.07489051,  0.49566702, -0.71639138, -1.28751712,
        -0.29678852, -0.72249044, -0.356725

In [26]:
# Преобразуем центры в список для сохранения
centers_list = [center.tolist() for center in centers]  # Преобразуем в список, если это необходимо

# Запись центров в JSON файл
with open("/content/cluster_centers.json", 'w') as json_file:
    json.dump(centers_list, json_file, indent=4)

# Задание 3. Регрессия (LinearRegression)

## 1. Сгенерировать подходящие исходные данные для проведения обучения

In [27]:
# Импорт датасета
from sklearn.datasets import make_regression

# Генерация датасета
X, y = make_regression(
    n_samples = 1000,
    n_features = 15,
    noise=1
)
# Запись датасета в файл
to_file("/content/regression", X, y)

In [28]:
# Загрузка данных в Spark
data = spark.read.format("libsvm").load("regression")
data.head()

Row(label=-108.68175590043617, features=SparseVector(15, {0: -1.0723, 1: 1.0257, 2: 0.9754, 3: -0.9159, 4: -0.9858, 5: 0.3526, 6: 0.2546, 7: -3.1075, 8: 0.4572, 9: -1.3914, 10: 1.2067, 11: 0.1114, 12: 0.1594, 13: 0.9043, 14: -0.1291}))

## 2. Провести кластеризацию с помощью pyspark.ml / pyspark.mllib

In [29]:
# Импорт модели
from pyspark.ml.regression import LinearRegression

# Разделение на обучающую и тестовую выборку
training, test = data.randomSplit([0.8, 0.2])

# Обучение
linear = LinearRegression()
model = linear.fit(training)

## 3. Провести расчет метрик

In [30]:
# Предсказание
predictions = model.transform(test)
predictions.head()

Row(label=-613.1225725552944, features=SparseVector(15, {0: -0.8259, 1: 0.3277, 2: 0.6235, 3: -1.4212, 4: 0.2434, 5: -1.1437, 6: -0.7456, 7: -0.6562, 8: 0.0359, 9: 0.3378, 10: -1.609, 11: -2.7415, 12: -2.2568, 13: 0.7665, 14: -0.6057}), prediction=-613.5452646843999)

In [34]:
# Коэффициенты модели
model.coefficients

DenseVector([0.0212, 22.5815, 61.698, 62.4909, 78.3392, 86.9825, -0.0021, 56.1446, 96.7084, -0.0019, -0.0229, 97.7877, 90.343, 19.7904, -0.0102])

In [35]:
# Свободный член
model.intercept

-0.03674295519504607

In [36]:
# Коэффициент детерминации
model.summary.r2

0.9999812710794743

In [37]:
# Оценка среднего квадратичного отклонения
model.summary.rootMeanSquaredError

1.013412803154909

## 4. Сохранить полученные значения метрик в файл.

In [38]:
# Подготовка данных для сохранения
metrics = {
    'Coefficients': model.coefficients.tolist(),  # Преобразуем в список, если необходимо
    'Intercept': model.intercept,
    'R²': model.summary.r2,
    'Root Mean Squared Error': model.summary.rootMeanSquaredError
}

# Запись метрик в JSON файл
with open("/content/model_metrics.json", 'w') as json_file:
    json.dump(metrics, json_file, indent=4)

# Задание 4. Word2vec

## 1. Сгенерировать подходящие исходные данные для проведения обучения

In [40]:
import random

# Список слов
words = ["word1", "word2", "word3", "word4", "word5", "word6"]

# Генерация текста
text = ' '.join(random.choices(words, k = 1000))

In [47]:
# Импорт модели
from pyspark.ml.feature import Word2Vec

# Генерация данных
data = spark.createDataFrame([(text.split(" "),)], ['text'])
data.show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## 2. Провести векторизацию слов с помощью pyspark.ml / pyspark.mllib

In [48]:
# Обучение
word2vec = Word2Vec(inputCol="text", outputCol="model")
model = word2vec.fit(data)

In [49]:
model.getVectors().show()

+-----+--------------------+
| word|              vector|
+-----+--------------------+
|word2|[0.00116086716298...|
|word5|[0.00255756359547...|
|word6|[0.00164286780636...|
|word3|[0.00722798146307...|
|word1|[0.00427288236096...|
|word4|[-0.0012193360598...|
+-----+--------------------+



## 3. Сохранить полученные векторы

In [50]:
import pandas as pd

# Преобразуем векторы в Pandas DataFrame для удобства
vectors_pd = model.getVectors().toPandas()

# Сохранение DataFrame в JSON файл
vectors_pd.to_json("/content/word2vec_vectors.json", orient='records', lines=True)